# A Gentle Introduction to the Linear Regression Model: The Mean Model

In [ ]:
# PACKAGES
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics as st
import statsmodels.api as sm

# FUNCTIONS FROM PACKAGES
from numpy.linalg import inv

# SEABORN THEME
scale = 0.4
W = 16*scale
H = 9*scale
sns.set(rc = {'figure.figsize':(W,H)})
sns.set_style("white")

Main References:
- For the part on the functions for the sample mean and variance, we took inspiration from the amazing class on [Multiple Equation Estimation](https://www.fionaburlig.com/teaching/are212) by Fiona Burlig (with R) at UC Berkeley.
- For more theory in statistics and econometrics, we relied on various sources on the Web and the following textbooks: 
    - J. Wooldridge, Econometric Analysis of Cross Section and Panel Data, MIT Press, 2002
    - William H. Greene, Econometric Analysis, sixth edition, Pearson.

Notation: 
- In the previous class [Probability and Statistics](https://github.com/edoardochiarotti/class_datascience/blob/main/Notebooks/Week_06/06_probability_and_statistics.ipynb) we have used capital letters, i.e. $X$, to denote random variables, and lowercase letter, i.e. $x$, to denote its numerical outcomes, or realizations.
- For this class, it's no longer convenient to keep that notation. We will thus denote both random variables and their realizations with lowercase letter, i.e. $x$ (or $x_i$). Then, we will denote N-dimensional column vectors (of random variables) with lower bold letters, i.e. $\boldsymbol{x} = (x_1,...,x_N)'$, and matrixes (of random variables) with capital bold letters, i.e. $\bf{X}$. We will try to stick to this notation as much as possible.

## Content
- [A Gentle Introduction to the Linear Regression Model: The Mean Model](#A-Gentle-Introduction-to-the-Linear-Regression-Model:-The-Mean-Model)
    - [Matrix Operations in Python](#Matrix-Operations-in-Python)
    - [The Multivariate Normal Distribution](#The-Multivariate-Normal-Distribution)
    - [Marginal and Conditional Probability Distributions](#Marginal-and-Conditional-Probability-Distributions)
    - [The Mean Model](#The-Mean-Model)
    - [The Mean Model in Matrix Notation](#The-Mean-Model-in-Matrix-Notation)
    - [The Mean Model in Python: CO2 Emissions per Capita](#The-Mean-Model-in-Python:-CO2-Emissions-per-Capita)

### Matrix Operations
- In this section we will quickly cover basic matrix operations in Python. For a review of matrix algebra using Python, you Students who can read this [QuantEcon article](https://datascience.quantecon.org/scientific/applied_linalg.html) on applied linear algebra, until "Inverse" included. A more advanced introduction to linear algebra by QuantEcon is [here](https://python.quantecon.org/linear_algebra.html).
- The **Python** package to work with vectors and matrixes is `Numpy`. Let's start creating some vectors and matrixes using numpy arrays:

In [ ]:
# matrixes
X1 = np.reshape(np.arange(6), (3, 2))
X2 = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
X3 = np.array([[2, 5, 2], [1, 2, 1]])
X4 = np.ones((2, 3))

# vectors
Y1 = np.array([1, 2, 3])
Y2 = np.array([0.5, 0.5])

- Let's now multiply 2 matrixes. For a refresher on **matrix multiplication**, you can check the page [Wikipedia, Matrix Multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication).
- For matrix multiplication, the number of columns in the first matrix must be equal to the number of rows in the second matrix. The resulting matrix, known as the matrix product, has the number of rows of the first and the number of columns of the second matrix. The product of matrices $\bf{A}$ and $\bf{B}$ is denoted as $\bf{AB}$.
- At the beginning, it's a good practice to always (always) write down the matrix dimensions as we write them down and we do operations with them. In python, you can display this information with `np.shape`:

In [ ]:
print(np.shape(X1)) # 3x2
print(np.shape(X4)) # 2x3

- OK it looks these two matrixes can be multiplied (the number of columns in the first matrix, i.e. 2, equals the number of rows in the second matrix, i.e. 2). Let's use the command `@` to multiply them, which is the same of `np.dot`:

In [ ]:
X1 @ X4

- The resulting matrix (np.array) is a 3-by-3 matrix, or matrix of size $3\times3$.
- We can also multiply a vector with a matrix:

In [ ]:
np.shape(Y1) # 1x3
np.shape(X1) # 3x2
Y1 @ X1

- Great. Let's now do the transpose of a matrix. In linear algebra, the **transpose** of a matrix is an operator which flips a matrix over its diagonal ([Wikipedia, Transpose](https://en.wikipedia.org/wiki/Transpose)).
- We can use the `.T` method of a `numpy.array`:

In [ ]:
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(X)
print(X.T)

- A matrix that we'll use a lot is the **identity matrix**. In linear algebra, the identity matrix of size $N$ is the $N\times N$ square matrix with ones on the main diagonal and zeros elsewhere ([Wikipedia, Identity Matrix](https://en.wikipedia.org/wiki/Identity_matrix)).
- When $\bf{A}$ is an $m\times n$ matrix, it is a property of matrix multiplication that $I_m\bf{A}=\bf{A}I_n=A$.
- In Python we can create identity matrixes with `np.eye`:

In [ ]:
I = np.eye(3)
X = np.reshape(np.arange(9), (3, 3))
Y = np.array([1, 2, 3])

print("I @ x", "\n", I @ X)
print("x @ I", "\n", X @ I)
print("I @ y", "\n", I @ Y)
print("y @ I", "\n", Y @ I)

- We'll also use a lot **matrix inversion**. An $N \times N$ square matrix $\bf{A}$ is called invertible (or nonsingular) if there exist an $N \times N$ square matrix $\bf{B}$ such that $\bf{A}B=BA=I$ ([Wikipedia, Invertible Matrix](https://en.wikipedia.org/wiki/Invertible_matrix)).
- In the theory of vector spaces, a set of vectors is said to be linearly dependent if there is a nontrivial linear combination of the vectors that equals the zero vector. If no such linear combination exists, then the vectors are said to be linearly independent ([Wikipedia, Linear Indipendence](https://en.wikipedia.org/wiki/Linear_independence)). In linear algebra, the rank of a matrix $\bf{A}$ is the maximal number of linearly independent columns of $\bf{A}$ ([Wikipedia, Rank](https://en.wikipedia.org/wiki/Rank_(linear_algebra))). If $\bf{A}$ is a $N \times N$ square matrix, then A is invertible if and only if A has rank $N$ (that is, A has full rank).
- The determinant is a scalar value that is a function of the entries of a square matrix. It allows characterizing some properties of the matrix and the linear map represented by the matrix ([Wikipedia, Determinant](https://en.wikipedia.org/wiki/Determinant)).
- Here is the formula for inverting a $2 \times 2$ square matrix $\bf{A}$:
<br><br>
$$
\boldsymbol{A}^{-1}=
\begin{bmatrix}
a & b \\
c & d 
\end{bmatrix}^{-1} = 
\frac{1}{|\boldsymbol{A}|}
\begin{bmatrix}
d & -b \\
-c & a 
\end{bmatrix} = 
\frac{1}{ad-bc}
\begin{bmatrix}
d & -b \\
-c & a 
\end{bmatrix}
$$

- In Python we can use the `numpy` function `linalg.inv`, which is here imported as `inv`:

In [ ]:
A = np.array([[2, 0], [3, 1]])

print("This is A inverse")
print(inv(A))

print("Check that A @ A inverse is I")
print(inv(A) @ A)

### The Multivariate Normal Distribution
- The **multivariate normal distribution**, multivariate Gaussian distribution, or joint normal distribution is a generalization of the one-dimensional (univariate) normal distribution to higher dimensions ([Wikipedia, Multivariate Normal Distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution)).
- The multivariate normal distribution is often used to describe, at least approximately, any set of (possibly) correlated real-valued random variables each of which clusters around a mean value.
- Let's take an 2-dimensional random vector $\boldsymbol{x}=(x_1, x_2)'$. Remember, $x_1$ is a random variable and $x_2$ is a random variable (here we are using lower capital letters for random variables, differently from the previous class, when random variables where always identified with capital letters).
- Let's write down the **probability density function** for the joint normal distribution:
<br><br>
$$
f(x_1,x_2)=\frac{1}{2\pi\sigma_1 \sigma_2 \sqrt{1-\rho^2}}exp\left(-\frac{1}{2(1-\rho^2)}\left[\left(\frac{x_1-\mu_1}{\sigma_1}\right)^2 -2\rho\ \left(\frac{x_1-\mu_1}{\sigma_1}\right) \left(\frac{x_2-\mu_2}{\sigma_2}\right) + \left(\frac{x_2-\mu_2}{\sigma_2}\right)^2 \right]\right)
$$
<br>
- We can write the same in matrix form:
<br><br>
$$
f(\boldsymbol{x})=\frac{1}{\sqrt{(2\pi)^N |\Sigma|}}\exp\left(-\frac{1}{2}(\boldsymbol{x}-\boldsymbol{\mu})'\boldsymbol{\Sigma}^{-1}(\boldsymbol{x}-\boldsymbol{\mu}))\right)
$$
<br>
- Stop for a moment. Write down these equations with pen and paper, and below each vector / matrix, write down its dimensions. For example, if a matrix has 2 columns and 2 rows, write below the matrix $2 \times 2$. As you do that, check that all dimensions allow matrix multiplication (i.e. matrixes are comformable). 
- The **moments** of the distribution are the following:
<br><br>
$$
\mathrm{E}(\boldsymbol{x})=(\mathrm{E}(x_1), \mathrm{E}(x_2))'=(\mu_1, \mu_2)'=\boldsymbol{\mu}
$$
<br>
$$
\mathrm{V}(\boldsymbol{x})=E[(\boldsymbol{x}-\boldsymbol{\mu})(\boldsymbol{x}-\boldsymbol{\mu})']= 
\begin{bmatrix}
\sigma_1^2 & \rho \\
\rho & \sigma_2^2 
\end{bmatrix} = 
\boldsymbol{\Sigma}
$$
<br>
- We can express all of this in the usual coincise notation:
<br>
$$
\boldsymbol{x} \sim \mathcal{N}(\boldsymbol{\mu},\,\boldsymbol{\Sigma})
$$
<br>

### Marginal and Conditional Probability Distributions
- It is important to distinguish between the joint probability distribution of $x_1$ and $x_2$ and the probability distribution of each variable individually. The individual probability distribution of a random variable is referred to as its **marginal probability distribution**. In general, the marginal probability distribution of $x_1$ can be determined from the joint probability distribution of $x_1$ and $x_2$. Let's write it down, and let's express its distributions and moments already in the coincise notation:
<br><br>
$$
f(x_1) = \int f(\boldsymbol{x})dx_2 = \frac{1}{\sigma_1\sqrt{2\pi}}\exp\left(-\frac{1}{2}\left(\frac{x_1-\mu_1}{\sigma_1}\right)^{2}\right)
$$
<br>
$$
x_1 \sim \mathcal{N}(\mu_1,\,\sigma_1^2)
$$
<br>
- Another imporatnt concept is the **conditional probability distribution**. Given two jointly distributed random variables $x_1$ and $x_2$, the conditional probability distribution of $x_1$ given $x_2$ is the probability distribution of $x_1$ when $x_2$ is known to be a particular value ([Wikipedia, Conditional Probability Distribution](https://en.wikipedia.org/wiki/Conditional_probability_distribution)). The conditional distribution contrasts with the marginal distribution of a random variable, which is its distribution without reference to the value of the other variable.
- Let's refer to $x_1$ conditional on $x_2$ as $x_1|x_2$. $x_1|x_2$ is a random variable, which has it own density function, i.e. the conditional density function, and its moments, which we can express with the usual coincise notation:
<br><br>
$$
f(x_1|x_2)=\frac{f(\boldsymbol{x})}{f(x_2)}=\frac{\frac{1}{\sqrt{(2\pi)^N |\Sigma|}}\exp\left(-\frac{1}{2}(\boldsymbol{x}-\boldsymbol{\mu})'\boldsymbol{\Sigma}^{-1}(\boldsymbol{x}-\boldsymbol{\mu}))\right)
}{\frac{1}{\sigma_2\sqrt{2\pi}}\exp\left(-\frac{1}{2}\left(\frac{x_2-\mu_2}{\sigma_2}\right)^{2}\right)}
$$
<br>
$$
x_1|x_2 \sim \mathcal{N}\left(\mu_1 + \frac{\sigma_1}{\sigma_2}\rho(x_2-\mu_2), \, (1-\rho)^2\sigma_1^2\right)
$$
<br>
- So far we have considered a random vector of 2 random variables. So what we have seen is more precisely the bivariate normal distribution and its marginals and conditionals. All these formulas can be **generalized**. 
- For example, when we have a N-dimensional random vector $\boldsymbol{x}=(x_1, x_2, ..., x_N)'$ that is normally distributed with mean $\boldsymbol{\mu}$ and variance-covariance matrix $\boldsymbol{\Sigma}$, we say that it follows a multivariate normal distribution $\boldsymbol{x} \sim \mathcal{N}(\boldsymbol{\mu},\,\boldsymbol{\Sigma}$. Note that the matrix notation we used it's already generalized, so (almost) all of the above holds for N-dimensional random vectors. For more notation on the Multivariate Normal Distribution, you can check [Wikipedia - Multivariate Normal Distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution).
- Also note that conditional distributions can be generalized for random vectors and matrixes (rather than random variables). For example we can have a N-dimensional random vector $\boldsymbol{\epsilon}$ and a $N \times M$ matrix $\boldsymbol{X}$, and we can express the random vector $\boldsymbol{\epsilon}$ conditional on the random matrix $\boldsymbol{X}$ as $\boldsymbol{\epsilon}|\boldsymbol{X}$.

### The Mean Model
- Let's reconsider our beloved **random sample**. Remember? Intuitively, you can think of the random sample as a representation of the sample before it is drawn. Each component is a random variable that expresses the probability that, when the sample is effectively drown, the respective component takes certain value / outcome / realization. As we said that each random variable of our random sample follows the distribution of the population, this probability is expressed by the distribution of the population (which is what statisticians use to represent the population). This distribution has its moments / parameters, which is what we aim to estimate with the use of estimators.
- In the previous class, we have used the notation $(X_1,X_2,...,X_N)$ to refer to a random sample, in which each random variable, i.e. $X_1$, $X_2$, ..., $X_N$ is indipendent and identically distributed (i.i.d.). For the distribution of these independent random variables, we have always assumed a Normal distribution.
- Now that we know matrix algebra, let's change notation. We'll write that our random sample is a N-dimensional random vector $\boldsymbol{x}=(x_1,x_2,...,x_N)'$, where $x_1,x_2,...,x_N$ are i.i.d. normally-distributed random variables. And since change is good, let's now start referring to our random variable of interest as $y_i$, with corresponding random sample $\boldsymbol{y}=(y_1,y_2,...,y_N)'$.
- OK. Let's now say that we assume that our random variable of interest $y_i$ is normally distributed with mean $\beta$ (another change in notation, before we were usually calling the mean $\mu$) and variance $\sigma^2$. We can say that this is how we **model** our variable, or more simply that this is our model. In coincise form:
<br><br>
$$
y_i \sim \mathcal{N}(\beta,\,\sigma^2) \qquad \text{for } i=1,...,N
$$ 
<br>
- Let us now re-write our model in linear regression form (same exact thing):
<br><br>
$$
y_i = \beta + \epsilon_i \qquad \text{with } \epsilon_i \sim \mathcal{N}(0,\,\sigma^2) \text{, for } i=1,...,N
$$
<br>
- This is what we can call **mean model**. We can say that we have split $y_i$ in its fixed mean component, $\beta$, and its random component, $\epsilon_i$. But the model is exactly the same, nothing has changed. It's just another way of expressing it.

### The Sample Mean and Variance
- As we have seen in the previous class, the **sample mean** is a good estimator for $\beta$. As change is really really good, let's now use the notation $\hat{\beta}$ to refer to the sample mean estimator (rather than $\overline{Y}$):
<br><br>
$$
\hat{\beta}=\frac{1}{N}\sum_{i=1}^{N}y_i
$$
<br>
$$
\hat{\beta} \sim \mathcal{N}(\beta, \, \frac{\sigma^2}{N})
$$
<br>
- Let's now define the predicted (or fitted) values and the residuals of our model as follows:
<br><br>
$$
\hat{y_i} = \hat{\beta} \qquad \text{for } i=1,...,N
$$
<br>
$$
\hat{u_i} = y_i - \hat{y_i} = y_i - \hat{\beta} \qquad \text{for } i=1,...,N
$$
<br>
- Great. Also from the previous class we know that the (population) **sample variance** is a good estimator for $\sigma^2$, which we can then use to estimate the standard error of the mean. Let's use $\hat{\sigma}^2$ to refer to the sample-variance estimator:
<br><br>
$$
\hat{\sigma}^2=\frac{1}{(N-1)}\sum_{i=1}^{N}(y_i-\hat{\beta})^2
$$
<br>
$$
\hat{\sigma}\frac{N-1}{\sigma} \sim \mathcal{\chi}^2_{N-1}
$$
<br>

### The Mean Model in Matrix Notation
- OK so we have our model and we have our estimators to estimate the population's parameters of interest. So far we have expressed our model in terms of our random variable $y_i$, which is a component of our random sample, which can be expressed as an N-dimensional vector of random variables $\boldsymbol{y}=(y_1,y_2,...,y_N)'$. Instead of expressing the model for each random variable and then write "for $i=1,...,N$" each time, we can express the model in terms of our random sample, using matrix notation. Let's define a N-dimensional vector of 1s $\boldsymbol{x}=(1,,...,1)'$ and write our model in **matrix form**:
<br><br>
$$
\boldsymbol{y} \sim \mathcal{N}(\boldsymbol{x}\beta, \, \sigma^2 \boldsymbol{I}_N)
$$
<br>
- Think for a second about what this is and means. We said that each random variable $y_i$ of the random sample follows a distribution, which is the one of the population (they are identically distributed). So when you take all the random variables $y_i$ together in a random sample, i.e. $(y_1,...,y_N)$ it's like having N distributions that are all the same, with same mean $\beta$ and variance $\sigma^2$. Yeah, indeed. So you have $N$ $betas$ and $N$ $sigmas$. 
- As before, we can express this model in regression form, as follows (with always $\boldsymbol{x}=(1,,...,1)'$):
<br><br>
$$
\boldsymbol{y} = \boldsymbol{x}\beta + \boldsymbol{\epsilon} \qquad \text{with } \boldsymbol{\epsilon} \sim \mathcal{N}(\boldsymbol{0}_N,\,\sigma^2\boldsymbol{I}_N)
$$
<br>
- The model we just wrote says that the random vector $\boldsymbol{y}$ follows a N-variate normal distribution, with **mean vector and variance-covariance matrix of the model** as follows:
<br><br>
$$
\mathrm{E}(\boldsymbol{y})= \mathrm{E}(\boldsymbol{x}\beta+\boldsymbol{\epsilon}) = \mathrm{E}(\boldsymbol{x}\beta) = \boldsymbol{x}\beta =
\begin{bmatrix}
1 \\
\vdots \\
1
\end{bmatrix}\beta=
\begin{bmatrix}
\beta \\
\vdots \\
\beta
\end{bmatrix}
$$
<br>
$$
\mathrm{V}(\boldsymbol{y}) = \mathrm{V}(\boldsymbol{x}\beta+\boldsymbol{\epsilon}) = \mathrm{V}(\boldsymbol{\epsilon}) = \sigma^2\boldsymbol{I}_N = \sigma^2
\begin{bmatrix}
1 & 0 & \dots & 0 \\
0 & 1 & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 1 \\
\end{bmatrix}=
\begin{bmatrix}
\sigma^2 & 0 & \dots & 0 \\
0 & \sigma^2 & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & \sigma^2 \\
\end{bmatrix}
$$
<br>
- The first **element** $\beta$ in $\mathrm{E}(\boldsymbol{y})$ is the mean (first moment) of the distribution of $y_1$. Similarly, the first **diagonal element** $\sigma^2$ of $\mathrm{V}(\boldsymbol{y})$ is the variance (second moment) of the distribution of $y_1$. Again, you can think that this distribution is like the distribution of the population. Following a similar logic, the last $\beta$ in $\mathrm{E}(\boldsymbol{y})$ is the mean of the distribution of $y_N$, and the last diagonal element $\sigma^2$ of $\mathrm{V}(\boldsymbol{y})$ is the variance of the distribution of $y_N$. Again, you can think that this distribution is like the distribution of the population.
- So you have **N distributions** that are like the distribution of the population. Which is the same of saying that you have N random variables that follow the distribution of the population. Let's keep thinking about N distributions rather than 1 distribution, it's easier to then understand the size of our matrixes.
- So, indeed, what is the **size** of the mean and variance-covariance matrixes? As you can see, $\mathrm{E}(\boldsymbol{y})$ is $N\times1$ and $\mathrm{V}(\boldsymbol{y})$ is $N\times N$. You should write these matrixes down with pen and paper, and write their sizes under each matrix. Only through that can you achieve a power greater than any Jedi (cit [Palpatine](https://medium.com/@Katha23/palpatine-only-through-me-can-you-achieve-a-power-greater-than-any-jedi-18ccf7a23f43)).
- Also note that the **off-diagonal elements** of $\mathrm{V}(\boldsymbol{y})$ are correlation coefficients, that express the correlation between two random variables $y_i$ and $y_j$, i.e. $\mathrm{Cov}(y_i,y_j)$ with $j\neq i$. As we are assuming that the variables are independent (aside from identically distributed), their correlation coefficients are all zero.

### The Sample Mean and Variance in Matrix Notation
- OK great. As we did for the LRM, we can also express our **estimators** for the mean and variance, and predicted values and residuals, in matrix form. Before checking what comes below, you should take a moment to think how we can do that. So, how can we express, say, the sample mean estimator $\frac{1}{N}\sum_{i=1}^{N}y_i$ in matrix form, knowing that we have defined the N-dimensional vector of 1s $\boldsymbol{x}=(1,...,1)'$? 
- Yeah we are gonna tell you. One way of doing it for sample mean, predicted values, residuals and variance is the following one:
<br><br>
$$
\hat{\beta}=\frac{1}{N}\sum_{i=1}^{N}y_i = (\boldsymbol{x}'\boldsymbol{x})^{-1}(\boldsymbol{x}'\boldsymbol{y})
$$
<br>
$$
\hat{\boldsymbol{y}}=\boldsymbol{x}\hat{\beta}
$$
<br>
$$
\hat{\boldsymbol{\epsilon}}=\boldsymbol{y} - \hat{\boldsymbol{y}}=\boldsymbol{y} - \boldsymbol{x}\hat{\beta}
$$
<br>
$$
\hat{\sigma}^2=\frac{1}{N-1}\sum_{i=1}^{N}(y_i-\hat{\beta})^2=\frac{(\boldsymbol{y}-\boldsymbol{x}\hat{\beta})'(\boldsymbol{y}-\boldsymbol{x}\hat{\beta})}{N-1}=\frac{\hat{\boldsymbol{\epsilon}}\,'\hat{\boldsymbol{\epsilon}}}{N-1}
$$
<br>
- You should take a moment to write down these equations and work out the results using your knowledge of matrix multiplication. For example, you should see why $(\boldsymbol{x}'\boldsymbol{x})=N$. As per usual, you should write the size of all vectors and matrixes below them. As you do that, you should notice that $\hat{\beta}$ and $\hat{\sigma}^2$ remain obviously one-dimensional random variables, which follow the usual univariate distributions (you can write down their distributions and moments with the usual coincise notation).
- OK so to **recap**, in the previous class we have introduced the concepts of population, random sample, the sample-mean and sample-variance estimators. If we make assumptions on the distribution of the population (i.e. it is normal with a certain mean and variance), that means that we are using a framework (model) to model our variable of interest. In this class, we have seen how we can express that model in regression form, and how we can use matrixes to have a compact notation for the random sample. After expressing our model in regression form with matrix notation, we have also expressed the sample-mean and sample-variance estimators in matrix notation. We have done so because this is the most general notation you can have, and it will be super handy when we do the OLS estimator, with whatever number of regressors.

### The Mean Model in Python
- Let's now use **Python** to write down the functions for the sample-mean and sample-variance estimators using these matrix forms.
- We'll test these functions on data from the **QoG Environmental Indicators dataset (QoG-EI)** (Povitkina, Marina, Natalia Alvarado Pachon & Cem Mert Dalli. 2021). The Quality of Government Environmental Indicators Dataset, version Sep21 (University of Gothenburg: [The Quality of Government Institute](https://www.gu.se/en/quality-government)), is a compilation of indicators measuring countries' environmental performance over time, including the presence and stringency of environmental policies, environmental outcomes (emissions, deforestation, etc.), and public opinion on the environment. Codebook and data are available [here](https://www.gu.se/en/quality-government/qog-data/data-downloads/environmental-indicators-dataset).
- Note that we could test theste functions also on generated data, but it's nicer to test them on real-life data. Let's get it:

In [ ]:
# get data
link = "https://www.qogdata.pol.gu.se/data/qog_ei_sept21.xlsx"
df_qog = pd.read_excel(link)

- For this exercise we'll use the sample mean to estimate the mean of CO2 emissions per capita. In QoG, the variable for total CO2 emissions per capita is `edgar_co2pc`:

In [ ]:
# indexes
indexes = ["ccodealp","year"]

# CO2 variables
variabs_co2 = ["edgar_co2pc"]

# subset
df_qog_sub = df_qog.loc[:,np.append(indexes,variabs_co2)]
df_qog_sub.head()

- OK good. As this is a panel dataset and we want to start simple, let's make it a cross section by taking the mean of all variables by country. Also, let's drop all missing values, as our sample mean estimator will work only on a clean dataset:

In [ ]:
# make cross section
df = df_qog_sub.groupby("ccodealp")[variabs_co2].mean().reset_index().dropna()
df

- Great! we have 178 countries, our index variable for the iso codes (`ccodealp`), which we keep as variable and not as index, and our variable for CO2 emissions `edgar_co2pc`.  
- So we want to estimate the mean of our variable of interest emissions per capita `edgar_co2pc`. Let's do the usual reasoning shall we? Our unit for this analysis are the countries. There is a population of countries in the world with their emissions, represented by a normal probability distribution of emissions with mean $\beta$ and variance $\sigma^2$ . We cannot work with the population, but we can work only with a sample of emissions for 178 countries we have at hand (which is the realization of a random sample). This sample is just a representation of the population of all countries in the world, which amounts to 196 countries (yeah, in this case our sample is very close to the full population, but still it's not the full population).
- The sample-mean estimator is a good estimator for the average emission $\beta$. As mentioned, if we define a N-dimensional vector of 1s $\boldsymbol{x}=(1,...,1)'$, the sample-mean estimator can be expressed as $\hat{\beta}= (\boldsymbol{x}'\boldsymbol{x})^{-1}(\boldsymbol{x}'\boldsymbol{y})$. So let's add this vector of ones to our dataset: 

In [ ]:
# put ones into data
df["ones"] = 1
df

- Great! We can now write our function for the sample mean estimator. As we need to work with matrixes, the first thing to do is to convert panda data into numpy arrays (what Python uses for matrixes). Once the conversion is done, we can write the formula of our estimator by using the functions for matrix operations (multiplication, transpose, inverse) that we have learned in Python.
- Let's give it a first try with a function that takes as arguments the dataframe, the name of our variable of interest (y) and the name of our single regressor, which is the vector of ones (X):

In [ ]:
# define sample mean function
def sample_mean_estimator(data, y, X):
    
    """ My Sample Mean Function """
    
    # store in matrixes
    ydata = data.loc[:,y].to_numpy()
    xdata = data.loc[:,X].to_numpy()

    # get sample mean
    betahat = (inv(xdata.T @ xdata)) @ (xdata.T @ ydata)

    # return
    return float(betahat)

- OK let's try it out:

In [ ]:
# try function
sample_mean_estimator(data = df, y = "edgar_co2pc", X = "ones")

- Aaaaargh! Fatal. Error. Right, let's try to understand what's wrong. The error is a "LinAlgError", so there must be something wrong with our linear algebra? In addition, the error says `0-dimensional array given. Array must be at least two-dimensional`. OK so we are feeding some function we have used with a 0-dimensional array, though the function wants an array that is at least two-dimensional. 
- Let's learn to debug. To debug a function, you first need to store the arguments you have used for the function as objects in the general environment in which you are working. Translated, that means store `df` as `data`, `y` as `"edgar_co2pc"` and X as `"ones"`. Then, copy below the these objects the body of your function and re-run:

In [ ]:
# debug
data = df
y = "edgar_co2pc"
X = "ones"

# store in matrixes
ydata = data.loc[:,y].to_numpy()
xdata = data.loc[:,X].to_numpy()

# get sample mean
betahat = (inv(xdata.T @ xdata)) @ (xdata.T @ ydata)

- OK same error. From the error, it looks like there can be something going on with matrix multiplication or inversion, as there is a problem with the dimension of some of our arrays (as you know, matrixes must be comformable to be multiplied, and matrixes must be square to be invertible). Let's try the first the multiplication `xdata.T @ xdata`:

In [ ]:
xdata.T @ xdata

- OK this works. Note that it's a scalar. It's our N, i.e. 178, because $(\boldsymbol{x}'\boldsymbol{x})=N$. So maybe that's why the inversion does not work ... . Let's now try it's inversion:

In [ ]:
inv(xdata.T @ xdata)

- We found the BUG!!!! Great let's now try to udnerstand how we can de-bug this bug. There is for sure a problem with the dimension of `xdata.T @ xdata` that `numpy.linalg.inv()`, our `inv()`, does not like. Maybe it does not work with scalars? Let's explore the object:

In [ ]:
# type of xdata.T @ xdata
issue = xdata.T @ xdata
type(issue)

- It's indeed a scalar, and more precisely a `numpy.int64` object, i.e. an integer. What happens if we put that integer into an array?

In [ ]:
solution = np.array([xdata.T @ xdata])
print(solution.shape)
inv(solution)

- Still does not work. Maybe it's because we have used a one dimensional array, while `inv()` works only with two dimensional arrays?

In [ ]:
solution = np.array([[xdata.T @ xdata]])
print(solution.shape)
print(inv(solution))

- YEAHHHH! OK so it looks that `inv()` can actually invert scalars, though they need to be contained within 2-dimensional arrays. Va bene (fair enough).
- One way to always impose 2 dimensions on arrays in Python is the numpy function `np.atleast_2d`. We can use this function directly on our xdata when it is a vector and not a matrix (if it was a matrix, there would not be any problem). Careful that this function produces row vectors, so since we are working with column vectors we always need to apply the transpose after this function. If we do all of this, then the matrix multiplication `xdata.T @ xdata` will automatically give 2-dimensional arrays. For par conditio, let's do it also for `ydata`, you never know if at some point we'll have to invert that one too.

In [ ]:
# define better sample mean function
def sample_mean_estimator(data, y, X):
    
    """ My Sample Mean Function """
    
    # store in matrixes
    ydata = data.loc[:,y].to_numpy()
    xdata = data.loc[:,X].to_numpy()
    
    # make column vectors for arrays with less than 2 dimensions
    if len(ydata.shape) == 1:
        ydata = np.atleast_2d(ydata).T
    if len(xdata.shape) == 1:
        xdata = np.atleast_2d(xdata).T

    # get sample mean
    betahat = (inv(xdata.T @ xdata)) @ (xdata.T @ ydata)

    # return
    return float(betahat)

- And let's re-run the function:

In [ ]:
sample_mean_estimator(data = df, y = "edgar_co2pc", X = "ones")

- Eureka! We have written a function for the sample-mean estimator using matrix algebra, and we have applied it to our data, ergo we just obtained our estimate for the mean of the population $\beta$. Our estimate is $5.01$ something.
- This is awesome! However, we could do better. True Python coders would tell you that an efficient function should not have more than 5 lines max. So to have a clean and efficient code, what you should do is split the function in mulptiple functions. There are many ways to do this, but we can follow our general logic. The first part of our function does not really do anything related to the estimator, it just transforms data into matrixes. In addition, as we'll work with matrixes a lot, it might be useful later on to have a stand-alone function that transforms data into matrixes. Makes sense. Let's do it:

In [ ]:
# define better functions

# function to transform panda series into vectors / matrices
def data_to_matrix(data, variab_name):
    
    """ My Data to Matrix Function """
    
    # store in matrixes
    matrix = data.loc[:,variab_name].to_numpy()
    
    # make column vectors for arrays with less than 2 dimensions
    if len(matrix.shape) == 1:
        matrix = np.atleast_2d(matrix).T
        
    # return result
    return matrix

# define sample mean function
def sample_mean_estimator(data, y, X):
    
    """ My Sample Mean Function """
    
    # store in matrixes
    ydata = data_to_matrix(data, variab_name = y)
    xdata = data_to_matrix(data, variab_name = X)

    # get sample mean
    betahat = (inv(xdata.T @ xdata)) @ (xdata.T @ ydata)

    # return
    return float(betahat)

- And re-run:

In [ ]:
# estimate sample mean and store
sample_mean = sample_mean_estimator(data = df, y = "edgar_co2pc", X = "ones")
sample_mean

- Same old same old? Not really. Yeah the result is the same (thank god), but your code is cleaner and easier to debug if there is a problem in the future. Cool.
- But are we sure our code is correct? How can we be sure? Easy, we compare it with a "canned" function for the sample mean provided by the package `statistics`, called `st.mean`, which we already used in the last class. As a note of caution, the formulas inside the function `st.mean` can be a bit different than our matrix formulas (you should look inside by using `??`). For this reason, the output might come out some decimals different. As that is the case and outputs will not be exactly exactly the same, `==` will not work. What we use in Python that 2 objects are essentially the same but not exactly, with a certain degree of approximation, is `np.allclose`.
- Let's do it:

In [ ]:
# do it with canned method and compare
sample_mean_canned = st.mean(df.loc[:,"edgar_co2pc"])
print(sample_mean_canned)
np.allclose(sample_mean, sample_mean_canned)

- BOOM! As you can see they are the same, but for the 15th decimal (so `==` would have returned `False`). Ok, for us this means that they are the same. We made it!!
- Let's now use the knowledge we have in building functions to make a nice function for the sample-variance estimator, following the formula we have written above in matrix notation:

In [ ]:
# write function for estimator for the standard error of the mean
def sample_variance_estimator(data, y, X, sample_mean):

    """ My Sample Variance Function """
    
    # store in matrixes
    ydata = data_to_matrix(data, variab_name = y)
    xdata = data_to_matrix(data, variab_name = X)

    # get params
    N = len(ydata)
    degrees_freedom = N-1

    # get sample variance
    resid = (ydata - np.dot(sample_mean, xdata))
    s2 = (resid.T @ resid) /degrees_freedom
    
    # return
    return float(s2)

- Let's try it:

In [ ]:
sample_variance = sample_variance_estimator(data = df, y = "edgar_co2pc", X = "ones", 
                                            sample_mean = sample_mean)
sample_variance

- In the same way, let's write a function for the estimator of the standard error of the mean. This would be simply the square root of the sample variance divided by N, but let's write a stand-alone function to do it, as we'll use it extensively later on:

In [ ]:
# write function for estimator for the standard error of the mean
def sem_estimator(data, y, X, sample_mean):

    """ My SEM Estimator Function """
    
    # store in matrixes
    ydata = data_to_matrix(data, variab_name = y)
    xdata = data_to_matrix(data, variab_name = X)

    # get params
    N = len(ydata)
    degrees_freedom = N-1

    # get sample variance
    resid = (ydata - np.dot(sample_mean, xdata))
    s2 = (resid.T @ resid) /degrees_freedom
    
    # get sem
    s = np.sqrt(sample_variance)
    sem = s / np.sqrt(N)
    
    # return
    return float(sem)

- And test:

In [ ]:
sem = sem_estimator(data = df, y = "edgar_co2pc", X = "ones", sample_mean = sample_mean)
sem

- OK now let's rely on the knowledge we have on statistical tests from the exercises of last class to write a function that does a z-score test on whether the estimate of the mean is significantly different from zero at the 5% level.
- Remember? In the exercises, we have done (i) z-test with known variance, with z-score following a standard normal distribution, (ii) t-test with unkown variance, with t-statistic following a T-student distribution with $N-1$ degrees of freedom. 
- Thanks to the Central Limit Theorem (which we saw last time), we know that, as the sample size increases, the sample-mean estimator $\hat{\beta}$ always follows a normal distribution. It turns out that, thanks to the same theorem, as the sample size increases the t-statistic always follows a standard normal distribution. Therefore, we can do a test with unknown variance using the functions and critical values of the normal distribution, such as `st.NormalDist()` and `stats.norm.ppf()`, rather than the functions and critical values of the t-student distribution (i.e. `stats.t.cdf()` and `stats.t.ppf()`).
- Let's write it:

In [ ]:
# write function for one sample t test statistic
def one_sample_z_test(sample_mean, sem, beta_null = 0, significance_level = 0.05):
    
    """ My Z Test Function """

    # get z score
    z_score = (sample_mean - beta_null) / sem

    # get p value
    lower_area = st.NormalDist().cdf(-z_score)
    upper_area = lower_area
    p_value = lower_area + upper_area

    # get confidence interval
    alpha_inv = (1.0-significance_level)
    q1 = (1+alpha_inv)/2
    ci_critical = stats.norm.ppf(q1)
    ci = (sample_mean-(ci_critical*sem), sample_mean+(ci_critical*sem))

    # return results
    return (z_score, p_value, ci)

- Aaand as usual, let's test it:

In [ ]:
# run and store z test
z_test = one_sample_z_test(sample_mean = sample_mean, sem = sem)
z_test

- And let's compare it with the results of the canned function `stats.weightstats.ztest` from the `statsmodels` package:

In [ ]:
# compare with canned methods
import statsmodels
z_test_canned = statsmodels.stats.weightstats.ztest(df["edgar_co2pc"])
print(z_test[0:2])
print(z_test_canned)
print(np.allclose(z_test[0:2], z_test_canned))

z_test_ci_canned = statsmodels.stats.weightstats.zconfint(x1 = df["edgar_co2pc"], x2=None, value=0, alpha=0.05)
print(z_test[2])
print(z_test_ci_canned)
print(np.allclose(z_test[2], z_test_ci_canned))

- A. MA. ZING.
- OK so to recap we have write functions for sample-mean, variance and standar-error estimators, and we have obtained estimates for all of these on a sample of CO2 emissions per capita at the country level. We have also run a statistical test to test whether the sample-mean estiamte is significantly different than 0 at the 5% confidence level, using both t statistic / p value and confidence interval. Let's display the results here all together in a compact way:

In [ ]:
# estimate sample mean and store
sample_mean = sample_mean_estimator(data = df, y = "edgar_co2pc", X = "ones")

# get p value of estimate being different than 0
sem = sem_estimator(data = df, y = "edgar_co2pc", X = "ones", sample_mean = sample_mean)
p_value = one_sample_z_test(sample_mean = sample_mean, sem = sem)[1]

# print
print("Estimate of Beta from Sample Mean Estimator:          ", round(sample_mean, 3))
print("P-value for Estimate of Beta being different than 0:  ", round(p_value, 3))

- Let's interpret these results. The coefficient 5.016 tells us that, on average, one person emits 5.016 of CO2 emissions per year. In addition, looking at the p-value of 0.000, we can say that this result is statistically significant at the 1% level (the p-value is smaller than 0.01). This means that, based on our sample and assumptions, we can be pretty darn sure that a person emits more than 0 tons of CO2 per year. The probability that we are wrong on this claim is 0.00000000000038846703634227 %.
- Next time we will do the same just in a bivariate regression model, and our coefficient will not be an estimate of the mean but an estimate of the relationship between a regressor and CO2 emissions per capita. All the concepts that we have seen today apply.